## Testiranje modela


In [2]:
from settings import *
from pprint import pformat
from fastai.vision.all import load_learner
import os


model_picks = next(os.walk("./models"))[1]
seen = set()
testees = {}


for model in model_picks:
    spot = Path(garage / model_picks[0])
    try: 
        spot = Path(garage / model)
        print(spot)
    except NameError:
        print(f"model was not defined. Going with {model_picks[0]}.")
    signature = pformat(spot)
    if signature not in seen:
        seen.add(signature)
        try:
            testees[model] = {
                "breath": load_learner(spot/"breath.pkl"),
                "depth": load_learner(spot/"depth.pkl")
            }
        except Exception as ex:
            print(f"Model {model} is not trained yet!")
            print(ex)

testees

models/googlenet_10x
models/googlenet_1x
models/googlenet_3x


{'googlenet_10x': {'breath': <fastai.learner.Learner at 0x7febf165dcd0>,
  'depth': <fastai.learner.Learner at 0x7febf0e6ae50>},
 'googlenet_1x': {'breath': <fastai.learner.Learner at 0x7febf0e928d0>,
  'depth': <fastai.learner.Learner at 0x7febf1691110>},
 'googlenet_3x': {'breath': <fastai.learner.Learner at 0x7febf0d20410>,
  'depth': <fastai.learner.Learner at 0x7febf0d4f410>}}

### Testiranje na bazi


In [4]:
import pandas as pd

df = pd.read_parquet("data/wikiart_composed.parquet")
df.head()

,artistName,title,year,style,genre,photography,artemis,emotions,Is painting,Face/body,Rating,artistContentId,artistUrl,paintingUrl,localUrl,webUrl,webUrl2
contentId,,,,,,,,,,,,,,,,,
314345,Hans von Aachen,Augustus and the Tiburtine Sibyl,1580,Mannerism (Late Renaissance),mythological painting,0,None,None,NaN,NaN,NaN,314332,hans-von-aachen,augustus-and-the-tiburtine-sibyl-1580,./wikiart/wikidata/images/hans-von-aachen/1580/314345.jpg,https://uploads6.wikiart.org/images/hans-von-aachen/augustus-and-the-tiburtine-sibyl-1580.jpg!Large.jpg,None
314363,Hans von Aachen,Portrait of Jacopo Biliverti,1585,Mannerism (Late Renaissance),portrait,0,None,None,NaN,NaN,NaN,314332,hans-von-aachen,portrait-of-jacopo-biliverti-1585,./wikiart/wikidata/images/hans-von-aachen/1585/314363.jpg,https://uploads2.wikiart.org/images/hans-von-aachen/portrait-of-jacopo-biliverti-1585.jpg!Large.jpg,None
314364,Hans von Aachen,Portrait of Joseph Heintz,1585,Mannerism (Late Renaissance),portrait,0,None,None,NaN,NaN,NaN,314332,hans-von-aachen,portrait-of-joseph-heintz-1585,./wikiart/wikidata/images/hans-von-aachen/1585/314364.jpg,https://uploads8.wikiart.org/images/hans-von-aachen/portrait-of-joseph-heintz-1585.jpg!Large.jpg,None
314371,Hans von Aachen,The rape of Proserpine,1587,Mannerism (Late Renaissance),mythological painting,0,None,None,NaN,NaN,NaN,314332,hans-von-aachen,the-rape-of-proserpine-1587,./wikiart/wikidata/images/hans-von-aachen/1587/314371.jpg,https://uploads5.wikiart.org/images/hans-von-aachen/the-rape-of-proserpine-1587.jpg!Large.jpg,None
314369,Hans von Aachen,The judgment of Paris,1588,Mannerism (Late Renaissance),mythological painting,0,None,None,NaN,NaN,NaN,314332,hans-von-aachen,the-judgment-of-paris-1588,./wikiart/wikidata/images/hans-von-aachen/1588/314369.jpg,https://uploads4.wikiart.org/images/hans-von-aachen/the-judgment-of-paris-1588.jpg!Large.jpg,None


In [5]:
from utils.tester import predict_columns

# prođi kroz bazu u malim batchevima i spremi u datoteke
dataset = df.loc[~df["artemis"].isna() | ~df["emotions"].isna()].sample(1000)
datafiles = {}
for model_name, testee in testees.items(): 
    datafiles[model_name] = predict_columns(testee, dataset, model_name, root);

UsageError: Line magic function `%%capture` not found.


In [ ]:
# spoji kreirane batch datoteke u jednu (ako je gore uzeti uzorak bio do 1000 jedinica, onda se ova čelija može preskočiti)
for model_name, addresses in datafiles.items():
    result = pd.DataFrame()
    for adress in addresses:
        data = pd.read_parquet(adress)
        result = pd.concat((result, data))

    result[["artistName", "title", "year", "style", "breath", "breath_probs", "depth", "depth_probs", "genre", "artemis", "emotions", "webUrl"]].to_parquet(f"data/{model_name}/critic_full.parquet")

In [5]:
# import pandas as pd

model_predictions = {}
for model in model_picks:
    model_predictions[model] = pd.read_parquet(Path("./data")/model/"critic_output1.parquet")
model_predictions[list(model_predictions.keys())[0]].sample(3)

,artistName,title,year,style,breath,breath_probs,depth,depth_probs,genre,artemis,emotions,webUrl
242169,Mark Rothko,Untitled,1952.0,Color Field Painting,abstract,1.000000,iconic,1.000000,abstract,"disgust, fear, excitement",,https://uploads5.wikiart.org/images/mark-rothko/untitled-6(1).jpg
224004,Pablo Picasso,Head of a man,1908.0,Cubism,abstract,0.998729,iconic,0.766468,portrait,"disgust, fear, something else",None,https://uploads5.wikiart.org/images/pablo-picasso/head-of-a-man-1908-1.jpg
291887,Horia Bernea,Simeze,NaN,Abstract Expressionism,abstract,0.996395,iconic,0.829505,abstract,"awe, contentment, something else, amusement",None,https://uploads8.wikiart.org/images/horia-bernea/simeze.jpg


### Testiranje na web stranicama


In [6]:
from utils.downloader import page_scraper
from utils.tester import test_learners

# "https://aiartists.org/generative-art-design"
# "https://unsplash.com"
# "https://www.metmuseum.org/art/collection/search?showOnly=openAccess"
# "https://www.gettyimages.com"

internet = [
    *page_scraper("https://www.gettyimages.com"),
]

# "https://tse2.mm.bing.net/th?id=OIP.UPZ1-G8gpc5FkNIC2RCWSgHaFj&pid=Api",
# "https://tse2.mm.bing.net/th?id=OIP.if_cidFAKZ49wY7BLA3feQHaGE&pid=Api",
# "https://tse4.mm.bing.net/th?id=OIP.Px4ySbgcqEgFJOhOq8k5mAHaEo&pid=Api",
# "https://tse4.mm.bing.net/th?id=OIP.eNfpYf9Oqyh0u3_b1Eu20wHaGL&pid=Api",
# "https://tse3.mm.bing.net/th?id=OIP.NEkbbYyu56hdqekgPKxmoQAAAA&pid=Api",
# "https://tse3.mm.bing.net/th?id=OIF.jw1Qziy4XGJbDtTYtrBY3Q&pid=Api",
# "https://tse2.mm.bing.net/th?id=OIP.pJwVOqij6rPFrOjcHc1jbAHaKj&pid=Api",
# "https://tse1.mm.bing.net/th?id=OIP.-Hy08hdnnfvCwor0Y_fyGAHaLA&pid=Api",
# "https://tse4.mm.bing.net/th?id=OIP.mUVhO-Zld2qEJKe_r9tJaAHaHn&pid=Api",
# "https://tse2.mm.bing.net/th?id=OIP.DqjuCxnocoFQ0hvIH_Cm8wHaEK&pid=Api",
# "https://tse1.mm.bing.net/th?id=OIP.EFoByYTKu1KbZgoVfocCDAHaFU&pid=Api",
# "https://tse2.mm.bing.net/th?id=OIP.BFRcQqZ5Bpw1a-6u_XoBrwHaFj&pid=Api",
# "https://tse2.mm.bing.net/th?id=OIP.2qqjQQ0wW37Lbvkx50zPxgHaEB&pid=Api"

for model_name, testee in testees.items():
    test_learners(testee, test_set=internet, model_name=model_name, root=root/"artefacts", preview=False)

['https://static.gettyimages.com/display-sets/creative-landing/images/GettyImages-2131911406.jpg', 'https://static.gettyimages.com/display-sets/creative-landing/images/GettyImages-1995304390.jpg', 'https://static.gettyimages.com/display-sets/creative-landing/images/GettyImages-2086233684.jpg', 'https://static.gettyimages.com/display-sets/creative-landing/images/GettyImages-1820574038.jpg', 'https://static.gettyimages.com/display-sets/creative-landing/images/GettyImages-2112002315.jpg', 'https://static.gettyimages.com/display-sets/creative-landing/images/GettyImages-1939317654.jpg', 'https://static.gettyimages.com/display-sets/creative-landing/images/GettyImages-2116292040.jpg', 'https://static.gettyimages.com/display-sets/creative-landing/images/GettyImages-1859049658.jpg', 'https://static.gettyimages.com/display-sets/creative-landing/images/GettyImages-2043042354.jpg', 'https://static.gettyimages.com/display-sets/creative-landing/images/GettyImages-2039536981.jpg', 'https://static.get

Image test1.jpg is concrete.
('abstract', 2.1008367e-05) ('concrete', 0.999979)


Image test1.jpg is iconic.
('iconic', 0.96346426) ('symbolic', 0.03653571)


#%&%##%&%##%&%##%&%##%&%##%&%##%&%##%&%##%&%##%&%##%&%##%&%##%&%##%&%##%&%#
- 2 - 2 - 2 - 2 - 2 - 2 - 2 - 2 - 2 - 2 - 2 - 2 - 2 - 2 - 2 - 


Image test2.jpg is concrete.
('abstract', 0.0002301997) ('concrete', 0.99976987)


Image test2.jpg is iconic.
('iconic', 0.99278766) ('symbolic', 0.007212331)


#%&%##%&%##%&%##%&%##%&%##%&%##%&%##%&%##%&%##%&%##%&%##%&%##%&%##%&%##%&%#
- 3 - 3 - 3 - 3 - 3 - 3 - 3 - 3 - 3 - 3 - 3 - 3 - 3 - 3 - 3 - 


...

Image test7.jpg is symbolic.
('iconic', 0.016993513) ('symbolic', 0.98300654)


#%&%##%&%##%&%##%&%##%&%##%&%##%&%##%&%##%&%##%&%##%&%##%&%##%&%##%&%##%&%#
- 8 - 8 - 8 - 8 - 8 - 8 - 8 - 8 - 8 - 8 - 8 - 8 - 8 - 8 - 8 - 


Image test8.jpg is concrete.
('abstract', 0.0003690807) ('concrete', 0.9996309)


Image test8.jpg is iconic.
('iconic', 0.71350574) ('symbolic', 0.28649423)


#%&%##%&%##%&%##%&%##%&%##%&%##%&%##%&%##%&%##%&%##%&%##%&%##%&%##%&%##%&%#
- 9 - 9 - 9 - 9 - 9 - 9 - 9 - 9 - 9 - 9 - 9 - 9 - 9 - 9 - 9 - 


Image test9.jpg is abstract.
('abstract', 0.999995) ('concrete', 5.060215e-06)


Image test9.jpg is symbolic.
('iconic', 0.23651478) ('symbolic', 0.76348525)


In [ ]:
import re 

html = 'aaa a ad <img image="tupo-_&$banana.jpeg 300w, milica0087Pilica.png 120w" /> safdsaf saf\
    \safdafa <img asf afasf srcset="data:image/bmp jJAJBDADHHANDKNDKAKNNDDMNANDMND" src="345a"/> \
    saf sa<img data-stretch="false" data-src="https://images.squarespace-cdn.com/content/v1/5c77350965a707ed1710a1bc/1588698548907-RADZYUVD1S7R2R7I0JMM/Generative+Software+Design+Example+-+Procedural+Modeling.jpeg" data-image="https://images.squarespace-cdn.com/content/v1/5c77350965a707ed1710a1bc/1588698548907-RADZYUVD1S7R2R7I0JMM/Generative+Software+Design+Example+-+Procedural+Modeling.jpeg" data-image-dimensions="1500x842" data-image-focal-point="0.5,0.5" alt="Generative Software Design Example - Procedural Modeling.jpeg" data-load="false" src="https://images.squarespace-cdn.com/content/v1/5c77350965a707ed1710a1bc/1588698548907-RADZYUVD1S7R2R7I0JMM/Generative+Software+Design+Example+-+Procedural+Modeling.jpeg" width="1500" height="842" alt="" sizes="100vw" style="display:block;object-fit: cover; width: 100%; height: 100%; object-position: 50% 50%" srcset="https://images.squarespace-cdn.com/content/v1/5c77350965a707ed1710a1bc/1588698548907-RADZYUVD1S7R2R7I0JMM/Generative+Software+Design+Example+-+Procedural+Modeling.jpeg?format=100w 100w, https://images.squarespace-cdn.com/content/v1/5c77350965a707ed1710a1bc/1588698548907-RADZYUVD1S7R2R7I0JMM/Generative+Software+Design+Example+-+Procedural+Modeling.jpeg?format=300w 300w, https://images.squarespace-cdn.com/content/v1/5c77350965a707ed1710a1bc/1588698548907-RADZYUVD1S7R2R7I0JMM/Generative+Software+Design+Example+-+Procedural+Modeling.jpeg?format=500w 500w, https://images.squarespace-cdn.com/content/v1/5c77350965a707ed1710a1bc/1588698548907-RADZYUVD1S7R2R7I0JMM/Generative+Software+Design+Example+-+Procedural+Modeling.jpeg?format=750w 750w, https://images.squarespace-cdn.com/content/v1/5c77350965a707ed1710a1bc/1588698548907-RADZYUVD1S7R2R7I0JMM/Generative+Software+Design+Example+-+Procedural+Modeling.jpeg?format=1000w 1000w, https://images.squarespace-cdn.com/content/v1/5c77350965a707ed1710a1bc/1588698548907-RADZYUVD1S7R2R7I0JMM/Generative+Software+Design+Example+-+Procedural+Modeling.jpeg?format=1500w 1500w, https://images.squarespace-cdn.com/content/v1/5c77350965a707ed1710a1bc/1588698548907-RADZYUVD1S7R2R7I0JMM/Generative+Software+Design+Example+-+Procedural+Modeling.jpeg?format=2500w 2500w" loading="lazy" decoding="async" data-loader="sqs">\
     ffasf asf asf <figure srcset= "https://pili?ćsss3245i.png" =src=><div style="background-image: url(http://vilinkonjic.jpeg )" >saf </div> \
        <img data-stretch="false" data-src="https://images.squarespace-cdn.com/content/v1/5c77350965a707ed1710a1bc/1588703003192-B8US29YF5NDQZQK6WPUN/Silk+Interactive+Generative+Art.jpg" data-image="https://images.squarespace-cdn.com/content/v1/5c77350965a707ed1710a1bc/1588703003192-B8US29YF5NDQZQK6WPUN/Silk+Interactive+Generative+Art.jpg" data-image-dimensions="1958x720" data-image-focal-point="0.5,0.5" alt="Silk Interactive Generative Art.jpg" data-load="false" src="https://images.squarespace-cdn.com/content/v1/5c77350965a707ed1710a1bc/1588703003192-B8US29YF5NDQZQK6WPUN/Silk+Interactive+Generative+Art.jpg" width="1958" height="720" alt="" sizes="100vw" style="display:block;object-fit: cover; width: 100%; height: 100%; object-position: 50% 50%" srcset="https://images.squarespace-cdn.com/content/v1/5c77350965a707ed1710a1bc/1588703003192-B8US29YF5NDQZQK6WPUN/Silk+Interactive+Generative+Art.jpg?format=100w 100w, https://images.squarespace-cdn.com/content/v1/5c77350965a707ed1710a1bc/1588703003192-B8US29YF5NDQZQK6WPUN/Silk+Interactive+Generative+Art.jpg?format=300w 300w, https://images.squarespace-cdn.com/content/v1/5c77350965a707ed1710a1bc/1588703003192-B8US29YF5NDQZQK6WPUN/Silk+Interactive+Generative+Art.jpg?format=500w 500w, https://images.squarespace-cdn.com/content/v1/5c77350965a707ed1710a1bc/1588703003192-B8US29YF5NDQZQK6WPUN/Silk+Interactive+Generative+Art.jpg?format=750w 750w, https://images.squarespace-cdn.com/content/v1/5c77350965a707ed1710a1bc/1588703003192-B8US29YF5NDQZQK6WPUN/Silk+Interactive+Generative+Art.jpg?format=1000w 1000w, https://images.squarespace-cdn.com/content/v1/5c77350965a707ed1710a1bc/1588703003192-B8US29YF5NDQZQK6WPUN/Silk+Interactive+Generative+Art.jpg?format=1500w 1500w, https://images.squarespace-cdn.com/content/v1/5c77350965a707ed1710a1bc/1588703003192-B8US29YF5NDQZQK6WPUN/Silk+Interactive+Generative+Art.jpg?format=2500w 2500w" loading="lazy" decoding="async" data-loader="sqs">'

src = "<(?:img|figure)[\s]*.*?(?:src|image)(?:set)?=[\"'\s]+((?:http)[^'\"\s]+(?:.jpg|.jpeg|.png|.gif|,))[^'\"]*?[\"']+.*?>"
bgurl = "url\([\s]*?((?:http)[^\"'\s]+(?:.jpg|.jpeg|.png))[^\"']*?\)"


re.findall(src, html, re.IGNORECASE), re.findall(bgurl, html, re.IGNORECASE)